# Tutorial to calculating SOAP distances

This tutorial illustrates the D-measure and SOAP distances and the literature search calculated for the article "Graph Similarity Drives Zeolite Diffusionless Transformations and Intergrowth". This implementation was made by Daniel Schwalbe-Koda. If you use this code or tutorial, please cite:

D. Schwalbe-Koda, Z. Jensen, E. Olivetti, and R. Gómez-Bombarelli. "Graph similarity drives zeolite diffusionless transformations and intergrowth." _Nature Materials_ (2019). Link: https://www.nature.com/articles/s41563-019-0486-1

## Imports

The calculations of this article were performed by using the [`soaplite` library](https://github.com/SINGROUP/SOAPLite), now part of the [`dscribe` library](https://github.com/SINGROUP/dscribe), by the group of Adam Foster. As we used the SOAPLite library, we will reproduce our steps by employing such library:

In [1]:
import numpy as np
import soaplite
from soaplite import genBasis
from ase.io import read

The work on SOAP representation and the SOAPLite library has been done by the following works:

* A. Bartók, R. Kondor, and G. Csányi. "On representing chemical environments". _Physical Review B_ **87**, 184115 (2013).
* S. De, A. Bartók, G. Csányi, and M. Ceriotti. "Comparing molecules and solids across structural and alchemical space". _Physical Chemistry Chemical Physics_ **18** (20), 13754 (2016).
* M. O. J. Jäger _et al_. "Machine learning hydrogen adsorption on nanoclusters through structural descriptors". _npj Computational Materials_ **4**, 37 (2018).

# Calculating the SOAP distance between two crystals:

### Reading the CIF files with ASE

In [2]:
GME = read('../data/cif/GME.cif')
AFI = read('../data/cif/AFI.cif')

### Generating the power spectrum

In our example, we are interested in generating the power spectrum of the zeolites we have. For this purpose, let us analyze the case of GME and AFI zeolites, both of which have CIF files provided in this tutorial.

We start by generating the basis functions with $r_\textrm{cut} = 10$ and radial basis size of 8, as explained in detail in the SOAPLite tutorial:

In [3]:
myAlphas, myBetas = genBasis.getBasisFunc(10.0, 8) # input: (rCut, NradBas)

Now generating the power spectrum for each one of the crystals using the hyperparameters $r_\textrm{cut} = 10$, radial basis size of 8 and $L_\textrm{max} = 5$:

In [4]:
spectrum_GME = soaplite.get_periodic_soap_structure(GME, myAlphas, myBetas, rCut=10.0, Lmax=5)
spectrum_AFI = soaplite.get_periodic_soap_structure(AFI, myAlphas, myBetas, rCut=10.0, Lmax=5)

### Average SOAP fingerprint

As explained in the methods section of our article, we use the average SOAP fingerprint for the zeolites (Eq. 7 of the paper):

In [5]:
p_GME = np.mean(spectrum_GME, axis=0)
p_AFI = np.mean(spectrum_AFI, axis=0)

The kernel distance between the structures is basically a matter of taking the inner product between the fingerprints and normalizing it (Eq. 9 of the paper):

In [6]:
K = np.dot(p_GME, p_AFI) / np.sqrt(np.dot(p_GME, p_GME) * np.dot(p_AFI, p_AFI))

Finally, we can calculate the The `SOAP_NORMALIZATION` factor is simply the normalization constant we used in our article to have the maximum SOAP distance between two zeolites equal to 1. The two zeolites with maximum SOAP distance are the pair BCT-RWY.

In [7]:
SOAP_NORMALIZATION = 0.41541

soap_dist = np.sqrt(2 - 2 * K) / SOAP_NORMALIZATION

Now, the final SOAP distance between GME and AFI:

In [8]:
print('SOAP distance between GME and AFI: %.4f' % soap_dist)

SOAP distance between GME and AFI: 0.1812


Which is the value we reported on the paper (up to a numerical error).